In [1]:
import numpy as np
import json
import pandas as pd
from scipy import interpolate
from gensim.models import KeyedVectors
import pickle
import re

from nltk.corpus import stopwords
from nltk import word_tokenize

from nltk import pos_tag
from string import punctuation,digits
import os
import pickle

from sklearn import preprocessing

import tensorflow as tf

from scipy.interpolate import interp1d

from keras.utils import to_categorical 


from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

C:\Users\simcy\Anaconda3\envs\tensorflow\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
Using TensorFlow backend.


In [2]:
a=[]
for i in list(punctuation):
    if i=='%':
        continue
    else:
        a.append(i)


In [3]:
def remove_punctuation(s):
    list_punctuation = a
    for i in list_punctuation:
        s = s.replace(i,'')
    return s

def clean_sentence(sentence):
    sentence = sentence.lower()
    sentence = re.sub(r'(\W)\1{2,}', r'\1', sentence) 
    sentence = re.sub(r'(\w)\1{2,}', r'\1\1', sentence)
    sentence = re.sub(r'^https?:\/\/.*[\r\n]*', r'', sentence)
    sentence = re.sub(r'(?P<url>http?://[^\s]+)', r'', sentence)
    sentence = re.sub(r"\@(\w+)", "", sentence)
    sentence = re.sub(r"$ ", "", sentence)
    sentence = sentence.replace('#',' ')
    sentence = sentence.replace("'s",' ')
    sentence = sentence.replace("-",' ')
    tokens = sentence.split()
    tokens = [remove_punctuation(w) for w in tokens]
    stop_words = set(stopwords.words('english'))
    tokens = [w for w in tokens if not w in stop_words]
    #remove_digits = str.maketrans('', '', digits)
    tokens = [w.translate(tokens) for w in tokens]
    tokens = [w.strip() for w in tokens]
    tokens = [w for w in tokens if w!=""]
    tokens = [i.replace("ocompany","OCOMPANY") for i in tokens]
    tokens = [i.replace("company","COMPANY") for i in tokens]

    tokens = ' '.join(tokens)
    return tokens

In [4]:
def extract_domain_specific_features(text):
    features=[]
    for i in text.split():
        match=re.search("(^|[ \t])([-+$]?(\d+|\.\d+|\d+\.\d*))($|[^+-.]*)", i)
        if  match:
            features.append(match.group(0))
    return features

    

In [5]:
print(extract_domain_specific_features("Hello $123 this"))

['$123']


In [6]:
typos={'NewsMorrisons':'News Morrissons',
       'Taylor Wimpey':'Tailor Wimpey',
       'AB InBev':'AB InBrev',
       'Aberdeen Asset Management':'Aberdeen Asset Managment',
       'International Business Machines Corp. ':'IBM',
      }

target_typos={
       'shoeders':'schroders',
     #  'jp morgan chase':'jpmorgan chase',
       'gs':'companiesg4s',
    ''
}


SyntaxError: invalid syntax (<ipython-input-6-b5ffe4864d86>, line 13)

In [125]:

def prepare_data(fname):
        
        with open(fname, encoding='utf-8') as f:
            foo = json.load(f)
            sentence_l=[]
            target_l=[]
            aspect_l=[]
            sentiment_l=[]
            snippet_l=[]
            features=[]
            for key in foo.keys():
                for info in foo[key]['info']:
                    #print(info)
                    
                    sentence=foo[key]['sentence']
                    z=sentence
                    target= info['target']
                
                    c=[]
                    #print(sentence)
                    sentence = sentence.replace("'s",' ')

                    for typo,correct in typos.items(): 
                        if typo in sentence:
                            
                            #print(sentence)

                            sentence=re.sub(typo,correct,sentence)
                            
                            
                            #print(sentence)
                            
                    #sentence=re.sub(target.lower(),'COMPANY',sentence)
                    #sentence = re.sub(r"[$]?(\w+)", "OCOMPANY", sentence)
                    

                    features.append(extract_domain_specific_features(sentence))
                    sentence=sentence.lower()
                
                    p=[]
                    for i in sentence.partition(target.lower()):
                        if i==target.lower():
                            p.append('COMPANY')
                            
                        else: 
                            p.append(i)
                    p=" ".join(p)
                    p=p.strip()
                    
                    l=[]
                    for i in p.split(" "):
                        if i.startswith('$'):
                            if not i.lstrip('$').isalpha():
                                l.append('')
                            elif not i.lstrip('$').isnumeric():
                                l.append("OCOMPANY")
                        else:
                            l.append(i)
                                
                    l=" ".join(l)
                    l=l.strip()            
                            
                    
                    sentence = [clean_sentence(x) for x in l.split(" ")]
                    #print(sentence)
                    sentence=' '.join(sentence)
                              
                    if 'COMPANY' not in sentence.split():
                        
                            k=[]
                            #print("STILL NOT SOLVED 1")
                            #print(sentence,"AND",target)
                            
                            #if sentence=='xli potential intermediate top uptrend charts tdf':
                            #      sentence=re.sub('xli','COMPANY',sentence)
                                    
                            new=target.lower()[:3]
                            for i in sentence.split():
                                #print(i)
                                if i.startswith(new):
                                    k.append('COMPANY')
                                else :
                                    k.append(i)
                                    
                            k=' '.join(k)
                            sentence=k
                            
                    #print(z,"AND",target)
                    #print(sentence)
                    #if 'COMPANY' not in sentence.split(" ") : 
                        #print("NOT SOLVED :",sentence, "   AND  ",target.lower())
                    #    sentence=re.sub('OCOMPANY','COMPANY',sentence)
                       # if 'COMPANY' in sentence.split(" ") : print("SOLVED :",sentence)
                        
                    if 'COMPANY' not in sentence.split(" "):
                    
                           # print("STILL NOT SOLVED 2")
                           # print(sentence,"AND",target)
                 
                            if sentence.strip().startswith('gemalto shares slump third profit'):
                                      sentence=re.sub('gemalto','COMPANY',sentence)
                                
                            if sentence.strip().startswith('coca cola shares rise earnings beat expectations'):
                                      sentence=re.sub('coca cola','COMPANY',sentence)
                                
                            if sentence.strip().startswith('alphabet revenue 21 percent'):
                                      sentence=re.sub('alphabet','COMPANY',sentence)                            

                                
                            if sentence.startswith('price comparison site slumps 4 y'):
                                     #sentence=re.sub('companiescoutts','companies coutts',sentence)
                                     sentence='COMPANY '+sentence
                            '''           
                            for typo,correct in target_typos.items(): 
                                    if typo ==target.lower():
                                            sentence=re.sub(correct,'COMPANY',sentence)   
                            '''                
                                    
                            #if 'COMPANY' not in sentence.split(" ") : 
                            #    sentence='COMPANY '+sentence+ " AND ALS0 "+ target
                            if 'COMPANY' not in sentence.split(" ") : print('NOT solved',z,"#####", target,"@@@@",sentence)
                
                    if sentence.strip().startswith('industry newsrevenue earnings take'):
                                sentence=re.sub('newsrevenue','news revenue',sentence)
                   
    
                    snippet=info['snippets'].lstrip('[')
                    snippet=snippet.rstrip(']')
                    snippet=snippet.lower()
                   
                    #print(snippet)
                    
                    sentiment_score = info['sentiment_score']
                                    #print(sentiment_score)
                    aspect= info['aspects']
                    
                    #print(aspect)
                    asp=aspect[0].lstrip("['")
                   
                    asp=asp.rstrip("']")
                    #print(asp)
                    l=asp.split("/")
                    #print(l)

                    aspect=l[1]
                    #print(aspect)
                        
                    sentence=re.sub(' +', ' ',sentence)
                    sentence=sentence.strip()
                    
                    
                    #print(sentence)
                    
                    sentence_l.append(sentence)
                    target_l.append(target)
                    sentiment_l.append(sentiment_score)
                    aspect_l.append(aspect)
                    s=snippet.lstrip('\'')
                    s=s.rstrip('\'')
                    #print(s)
                    snippet_l.append(s)




            
        return sentence_l,target_l,sentiment_l,aspect_l,features,snippet_l

print("preparing Finance dataset...")
fname = {
            'finance': {
                'train_headline': r'D:\Datasets\FinanceHeadlineDataset\train\headline.json',
                'train_post': r'D:\Datasets\FinanceHeadlineDataset\train\post.json',
                'test_headline':  r'D:\Datasets\FinanceHeadlineDataset\test\headline.json',
                'test_post':  r'D:\Datasets\FinanceHeadlineDataset\test\post.json',
                'validation_post' : r'D:/PythonCodes/Sentiment-Analysis/Data/validation_post_1.json',
                'validation_headline' :r'D:/PythonCodes/Sentiment-Analysis/Data/validation_headline_1.json'
            },
            'Data_Augmentation':{
                'sentiment':{
                    'train_headline':r'D:\PythonCodes\Sentiment-Analysis\Data\Headline_Train.json',
                    'train_post':r'D:\PythonCodes\Sentiment-Analysis\Data\Microblog_Trainingdata.json',
                    'validation_headline':'D:\PythonCodes\Sentiment-Analysis\Data\Headline_Trialdata.json',
                    'validation_post':'D:\PythonCodes\Sentiment-Analysis\Data\Microblog_Trialdata.json'
                },
                'aspect':{
                    'train_data':'train_data_augmented.dat'
                }
            }


        }


preparing Finance dataset...


#  TRAIN 

In [77]:
#headlines=pd.DataFrame(data={'sentence':h_sentence,'sentiment':h_sentiment,'aspect':h_aspect,'snippet':h_snippet,'features':h_features})
#pickle.dump(headlines,open('headlines.data',"wb"))

In [7]:
headlines=pickle.load(open('headlines.data',"rb"))


In [8]:
headlines.head()

,aspect,features,sentence,sentiment,snippet
0,Rumors,[],ab inbrev latest bid said unlikely win COMPANY...,0.127,latest bid said unlikely to win sabmiller's ap...
1,Appointment,[],COMPANY shakes board two new business chiefs t...,-0.074,"shakes up board with two new business chiefs, ..."
2,Appointment,[],press serco set appoint roy gardner ex COMPANY...,0.195,"set to appoint roy gardner, ex-centrica, as ch..."
3,Appointment,[],press COMPANY set appoint roy gardner ex centr...,0.122,"set to appoint roy gardner, ex-centrica, as ch..."
4,Strategy,[],barclays sells benchmark indices unit COMPANY,0.281,sells benchmark indices unit


In [12]:
post=pickle.load(open('post.dat',"rb"))
post.iloc[650]

aspect                                              Sales
features                                            [-4%]
sentence     COMPANY gap september comparable store sales
sentiment                                          -0.441
snippet              september comparable store sales -4%
Name: 650, dtype: object

In [51]:
#h_sentence,h_target,h_sentiment,h_aspect,h_snippet,h_features=prepare_data(fname['finance']['train_headline'])


In [52]:
#p_sentence,p_target,p_sentiment,p_aspect,p_snippet,p_features=prepare_data(fname['finance']['train_post'])

In [21]:
#p_sentence[:1],p_target[:1]

In [ ]:
#post=pd.DataFrame(data={'sentence':p_sentence,'sentiment':p_sentiment,'aspect':p_aspect,'snippet':p_snippet,'features':p_features})
#pickle.dump(post,open('post.dat',"wb"))

In [ ]:
for i in p_sentence:
    h_sentence.append(i)

for i in p_sentiment:
    h_sentiment.append(i)

for i in p_aspect:
    h_aspect.append(i)
    
for i in p_snippet:
    h_snippet.append(i)

In [ ]:
train_data=pd.DataFrame(data={'sentence': h_sentence,'sentiment':h_sentiment,'aspect':h_aspect,'snippet':h_snippet})
pickle.dump(train_data,open("train_data_initial.dat","wb"))

In [120]:
train_data.head()

NameError: name 'train_data' is not defined

In [ ]:
p_sentence[:2],p_target[:2],p_sentiment[:2],p_aspect[:2],p_snippet[:2],p_features[:2]

# TEST

In [109]:
t_h_sentence,t_h_target,t_h_sentiment,t_h_aspect,t_h_features,t_h_snippet=prepare_data(fname['finance']['validation_headline'])

In [111]:
head=pd.DataFrame(data={'sentence':t_h_sentence,'sentiment':t_h_sentiment,'aspect':t_h_aspect,'snippets':t_h_snippet})
pickle.dump(head,open('test_head.dat',"wb"))

In [129]:
post=pd.DataFrame(data={'sentence':t_p_sentence,'sentiment':t_p_sentiment,'aspect':t_p_aspect,'snippets':snippet})
pickle.dump(post,open('test_post.dat',"wb"))

In [126]:
t_p_sentence,t_p_target,t_p_sentiment,t_p_aspect,t_p_features,snippet=prepare_data(fname['finance']['validation_post'])

In [128]:
t_p_sentence

['COMPANY currently way undervalued rise back soon',
 'COMPANY q217 delivered 6% organic revenue de growth 3% reported net revenue de OCOMPANY q2 organic revenue 31% reported net revenue 20%',
 'one stock oversold rsi 20 watch possible reverseal COMPANY',
 'COMPANY added dip yesterday rick holding night i’m running away',
 'COMPANY revenue falls 9% lowers full year revenue guidance',
 'COMPANY 25k shares left u might sell monday',
 'COMPANY shares slump third profit warning … via',
 'COMPANY revenue growth 3% past year 20th consecutive yoy decline OCOMPANY',
 'COMPANY downgraded neutral buy goldman pt',
 'COMPANY 87% analysts buy rating OCOMPANY 20% upside',
 'COMPANY sell pop market extreme overbought OCOMPANY OCOMPANY OCOMPANY',
 'COMPANY ca na bk analyst dropped rating perform target price high likelihood dividend cut',
 'COMPANY closed 8ema sold position',
 'rsi overbought COMPANY momentum falling',
 'COMPANY stock oversold shorted death dont really think matters er says shorts mon

In [101]:
headlines = pickle.load(open('test_head.dat',"rb"))

In [108]:
test_sentence=[]
test_sentiment=[]
test_aspect=[]

In [102]:
test_sentence,test_sentiment,test_aspect=headlines['sentence'],headlines['sentiment'],headlines['aspect']

In [109]:
for i in headlines['sentence']:
    test_sentence.append(i)

for i in headlines['sentiment']:
    test_sentiment.append(i)

for i in headlines['aspect']:
    test_aspect.append(i)

In [112]:
len(test_sentence)

192

In [111]:
for i in t_p_sentence:
    test_sentence.append(i)

for i in t_p_sentiment:
    test_sentiment.append(i)

for i in t_p_aspect:
    test_aspect.append(i)

In [113]:
Validation_data=pd.DataFrame(data={'sentence': test_sentence,'sentiment':test_sentiment,'aspect':test_aspect})

In [114]:
Validation_data.head()

,aspect,sentence,sentiment
0,Appointment,COMPANY uk personnel director quits supermarket,-0.223
1,Strategy,companies COMPANY raises â£ ahead london listing,0.296
2,Risks,COMPANY chief warns squeeze high street retailers,-0.397
3,Strategy,COMPANY backs view buys businesses,0.487
4,Financial,COMPANY trading profit beats expectations,0.565


In [115]:
pickle.dump(Validation_data,open("Final_ValidationData.dat","wb"))

In [116]:
t_p_sentence[:1],t_p_target[:1],t_p_sentiment[:1],t_p_aspect[:1],t_p_features[:1]

(['cantor fitzgerald sticks buy rating amazon COMPANY'],
 ['AMZN'],
 [0.247],
 ['Coverage'],
 [[]])

# DATA AUGMENT

In [65]:
h_sentence_1=[]
h_target_1=[]
h_sentiment_1=[]

p_sentence_1=[]
p_target_1=[]
p_sentiment_1=[]


def prepare_data1(fname):
        with open(fname, encoding='utf-8') as f:
            foo = json.load(f)
            sentence_l=[]
            target_l=[]
            sentiment_l=[]
            features=[]
            for info in foo:
                sentence=info['title']
    
                target= info['company']
                features.append(extract_domain_specific_features(sentence))
                    
                #print(sentence," +Tar+ ",target)

                m=[]
                for i in sentence.partition(target):
                        if i==target:
                            m.append('COMPANY')

                        else : m.append(i)
                m=' '.join(m)
                k=[]
                for i in m.split():

                    if i.lstrip('$')==target[0]:
                            k.append('COMPANY')

                    elif i ==target[0]:
                            k.append('COMPANY')

                    else: 
                        k.append(i)
                k=' '.join(k)

            
                sentence = [clean_sentence(x) for x in k.split(" ")]
                            #print(sentence)
                sentence=' '.join(sentence)

                sentiment_score = info['sentiment']
                                #print(sentiment_score)
           
                sentence=re.sub(' +', ' ',sentence)
                sentence=sentence.strip()

                sentence_l.append(sentence)
                target_l.append(target)
                sentiment_l.append(sentiment_score)
                



            
        return sentence_l,target_l,sentiment_l,features

h_sentence_1,h_target_1,h_sentiment_1,h_features_1=prepare_data1(fname['Data_Augmentation']['sentiment']['validation_headline'])


In [66]:
h_sentence_1[:5],h_target_1[:5],h_sentiment_1[:5],h_features_1[:5]

(['COMPANY says uk store closures put jobs risk',
  'COMPANY concrete bid holcim lafarge assets',
  'crh concrete bid COMPANY assets',
  'COMPANY share price slides underwhelming full year results',
  'COMPANY bid mr bricolage runs trouble'],
 ['Tesco', 'CRH', 'Holcim Lafarge', 'Reed Elsevier', 'Kingfisher'],
 [-0.9, 0.3, 0.3, -0.9, -0.3],
 [['2000'], [], [], [], []])

In [67]:
h_sentence_2,h_target_2,h_sentiment_2,h_features_2=prepare_data1(fname['Data_Augmentation']['sentiment']['train_headline'])


In [68]:

def prepare_data2(fname):
        with open(fname, encoding='utf-8') as f:
            foo = json.load(f)
            sentence_l=[]
            target_l=[]
            sentiment_l=[]
            features=[]
            for info in foo:
                sentence=info['spans'][0]
                #print(sentence)
                target= info['cashtag']
                
                features.append(extract_domain_specific_features(sentence))
                
                m=[]
                for i in sentence.partition(target):
                        if i==target:
                            m.append('COMPANY')

                        else : m.append(i)
                m=' '.join(m)
               
                #target= info['cashtag'].lstrip("$")
                #print(target)

                sentiment_score = info['sentiment score']
                #print(sentiment_score)
        
                sentence_l.append(m)
                target_l.append(target)
                sentiment_l.append(sentiment_score)
        return sentence_l,target_l,sentiment_l,features
                
p_sentence_1,p_target_1,p_sentiment_1,p_features_1=prepare_data2(fname['Data_Augmentation']['sentiment']['validation_post'])


In [69]:
p_sentence_1[:1],p_target_1[:1],p_sentiment_1[:1],p_features_1[:1]

(['Putting on a little  COMPANY  short'], ['$F'], [-0.454], [[]])

In [70]:
p_sentence_2,p_target_2,p_sentiment_2,p_features_2=prepare_data2(fname['Data_Augmentation']['sentiment']['train_post'])


In [71]:
len(h_sentence), len(p_sentence)

(498, 675)

In [72]:
len(h_sentence_1), len(p_sentence_1)

(14, 10)

In [73]:
len(h_sentence_2), len(p_sentence_2)

(1142, 1700)

# Combine ALL

In [74]:
#Sentence
for i in h_sentence_1:
    h_sentence.append(i)
for i in h_sentence_2:
    h_sentence.append(i)
    
for i in p_sentence_1:
    p_sentence.append(i)
for i in p_sentence_2:
    p_sentence.append(i)

#Sentiment
for i in h_sentiment_1:
    h_sentiment.append(i)
for i in h_sentiment_2:
    h_sentiment.append(i)

for i in p_sentiment_1:
    p_sentiment.append(i)
for i in p_sentiment_2:
    p_sentiment.append(i)
    
#Features
for i in h_features_1:
    h_features.append(i)
for i in h_features_2:
    h_features.append(i)

for i in p_features_1:
    p_features.append(i)
for i in p_features_2:
    p_features.append(i)

In [75]:
len(h_sentence), len(p_sentence)

(1654, 2385)

In [76]:
for i in p_sentence:
    h_sentence.append(i)
for i in p_sentiment:
    h_sentiment.append(i)
for i in p_features:
    h_features.append(i)

In [77]:
len(h_sentence)

4039

In [78]:
data={'sentence':h_sentence,
     'sentiment':h_sentiment,
     'features':h_features}
pickle.dump(data,open("Final_TrainData.dat","wb"))

In [79]:
data=pickle.load(open("Final_TrainData.dat","rb"))

In [80]:
len(data['sentence'])

4039

# Word Embedding

In [36]:
def load_google_word2vec(file_name):
    return KeyedVectors.load_word2vec_format(file_name, binary=True)


def build_embedding_matrix(vocab_size, embed_dim, tokenizer ):
    
    embedding_matrix_file_name='D:/PythonCodes/Sentiment-Analysis/Data/embedding_matrix_sentiment.dat'
    
    if os.path.exists(embedding_matrix_file_name):
        print('loading embedding_matrix:', embedding_matrix_file_name)
        embedding_matrix = pickle.load(open(embedding_matrix_file_name, 'rb'))
   
    else:
        print('loading word vectors...')
        fname = 'D:\PythonCodes\Jupyter notebooks\Word Embeddings\GoogleNews-vectors-negative300.bin' 

        model=load_google_word2vec(fname)

        embedding_matrix = np.zeros((vocab_size, embed_dim))

        for word, i in tokenizer.word_index.items():

            try:
                embedding_vector = model[word]
            except KeyError:
                embedding_vector = None
            if embedding_vector is not None:
                embedding_matrix[i]=embedding_vector


        pickle.dump(embedding_matrix, open(embedding_matrix_file_name, 'wb'))


    return embedding_matrix


In [43]:
embedding_matrix_file_name='D:/PythonCodes/Sentiment-Analysis/Data/embedding_matrix_sentiment.dat'
    
if os.path.exists(embedding_matrix_file_name):
    print('loading embedding_matrix:', embedding_matrix_file_name)
    embedding_matrix = pickle.load(open(embedding_matrix_file_name, 'rb'))
len(embedding_matrix)

loading embedding_matrix: D:/PythonCodes/Sentiment-Analysis/Data/embedding_matrix_sentiment.dat


5357

# SENTIMENT RESCALING

In [44]:
def rescale(series,old_range,new_range):
    m = interp1d(old_range,new_range)
    return [float(m(x)) for x in series]

In [4]:
sentiment = rescale(data['sentiment'],[-1,1],[0,1])

NameError: name 'rescale' is not defined

In [46]:
head_sentiment = rescale(t_h_sentiment,[-1,1],[0,1])

In [47]:
post_sentiment = rescale(t_p_sentiment,[-1,1],[0,1])

# TOKENIZING

In [48]:
def create_tokenizer(lines):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(lines)
    return tokenizer

# encode a list of lines
def encode_text(tokenizer, lines, length):
    encoded = tokenizer.texts_to_sequences(lines)
    padded = pad_sequences(encoded, maxlen=length, padding='post')
    return padded

In [49]:
import random
c = list(zip(data['sentiment'], data['sentence'],data['features']))
random.shuffle(c)
data['sentiment'], data['sentence'],data['features']= zip(*c)

In [50]:
tokenizer = create_tokenizer(data['sentence'])
vocab_size = len(tokenizer.word_index) + 1

In [51]:
vocab_size

5357

In [84]:
data['sentence'][:10]

('Bad Apple  ',
 'intertek profit expects meet forecast',
 'Reuters: Green Mountain revenue misses, shares plunge http://stks.co/13mW > $ COMPANY prints 43.80, market in a foul mood, bad day to disappoint',
 'Morrisons and COMPANY surprise City with Christmas bounce back',
 'In Play Longs  ',
 'aldi lidl expansion plans speed ahead tesco COMPANY morrisons',
 'Monday sold  ',
 'helge lund moves COMPANY month early',
 'Drugs and Biotech Getting Perky  ',
 'long  ')

In [90]:
target=[]
for i in data['sentence'] :
    if "COMPANY" in i :
        target.append('COMPANY')
    else:
        target.append("")


In [77]:
data['sentence'][5]
    

'aldi lidl expansion plans speed ahead tesco COMPANY morrisons'

In [78]:
trainX[5]

array([1227, 1228,  400,  215,  588,  166,   47,    1,  345,    0,    0,
          0,    0,    0,    0])

In [91]:
max_length=15
trainX= encode_text(tokenizer, data['sentence'], max_length)
target= encode_text(tokenizer, target, 1)
trainY= sentiment

In [92]:
target=np.tile(target,15)

In [54]:
embedding_matrix = build_embedding_matrix(vocab_size, 300,tokenizer)

loading embedding_matrix: D:/PythonCodes/Sentiment-Analysis/Data/embedding_matrix_sentiment.dat


In [55]:
len(eval('embedding_matrix'))

5357

In [56]:
head_X= encode_text(tokenizer, t_h_sentence, max_length)
postX =encode_text(tokenizer, t_p_sentence, max_length)

In [5]:
data_augmented_test={'data':data}

In [6]:
pickle.dump(data_augmented_test,open("Sentiment.dat","wb"))

In [93]:
data_for_sentiment={
    'trainX': trainX,
    'target':target,
    'trainY':trainY,
    'trainFeatures':h_features,
    'HEAD_testX':head_X,
    'HEAD_testY':head_sentiment,
    'POST_testX':postX,
    'POST_testY':post_sentiment,
    'embedding_matrix':embedding_matrix,
    'vocab_size':vocab_size
}

In [94]:
pickle.dump(data_for_sentiment,open("ALLdataForSentiment.dat","wb"))